In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import psycopg2

In [3]:
import numpy as np
import pandas as pd
import re

In [4]:

conn = psycopg2.connect("dbname=top100 user=postgres password=im4uema3cohb")

In [57]:
cur = conn.cursor()
command = "DROP TABLE IF EXISTS lyrics;"
cur.execute(command)
command = "DROP TABLE IF EXISTS music;"
cur.execute(command)
command = "DROP TABLE IF EXISTS artist;"
cur.execute(command)
cur.close()

In [58]:
#table paroles avec foreign key nom musique et nom artiste
commands = (
        """
CREATE TABLE artist(
   id_artist INT GENERATED ALWAYS AS IDENTITY,
   nameArtist VARCHAR(100),
   birth VARCHAR(100),
   city VARCHAR(100),
   country VARCHAR(100),
   PRIMARY KEY(id_artist)
)
        """,
        """
CREATE TABLE music(
   id_music INT GENERATED ALWAYS AS IDENTITY,
   id_artist INT,
   title VARCHAR(100),
   genre VARCHAR(100),
   releaseDate INT,
   ranking INT,
   month VARCHAR(100),
   PRIMARY KEY(id_music),
   CONSTRAINT fk_artist
      FOREIGN KEY(id_artist) 
	  REFERENCES artist(id_artist)
	  ON DELETE CASCADE
)
        """,
        """
CREATE TABLE lyrics(
   id_lyrics INT GENERATED ALWAYS AS IDENTITY,
   id_artist INT,
   id_music INT,
   lyrics VARCHAR(1000),
   PRIMARY KEY(id_music),
   CONSTRAINT fk_artist
      FOREIGN KEY(id_artist) 
	  REFERENCES artist(id_artist)
	  ON DELETE CASCADE,
   CONSTRAINT fk_music
      FOREIGN KEY(id_music) 
	  REFERENCES music(id_music)
	  ON DELETE CASCADE
)
        """)


In [59]:
cur = conn.cursor()
# create table one by one
for command in commands:
        cur.execute(command)
# close communication with the PostgreSQL database server
cur.close()
# commit the changes
conn.commit()

In [25]:
if conn is not None:
    conn.close()

In [4]:
import os
import pandas as pd

In [6]:
dfs_chart[2004]["Juillet"]

,Rank,Artist,Music,Naissance,Pays d'origine,Origine,Nationalité,Pays
0,1,O-zone,Dragostea Din Tei,,Moldavie,,,Moldavie
1,2,K'maro,Femme Like U,,Canada,,,Canada
2,3,Calogero,Face À La Mer,"30 juillet 1971 (51 ans)Échirolles, France",,,Française,
3,4,T-rio,Mamae Eu Quero (choopeta),,,,,
4,5,Le 6-9,Le Poulailler,,,,,"France, Suisse"
...,...,...,...,...,...,...,...,...
95,96,Sandra Lou,Le Banana Split,,,,Française,
96,97,Mickey 3d,Johnny Rep,,France,,,France
97,98,Jean-luc Lahaye,On N'se Reverra Plus,"23 décembre 1952 (69 ans)Paris, France",,,,
98,99,Lmc,Take Me To The Clouds Above,,,,,


In [78]:

dfs_chart[2020]["Fevrier"][dfs_chart[2020]["Fevrier"]['Naissance'].str.contains("10 mai 1995")]

,Rank,Artist,Music,Naissance,Pays d'origine,Origine,Nationalité,Pays
11,12,Aya Nakamura,40%,10 mai 1995 ou 10 mai 1995Bamako,,,Malienne,
69,70,Aya Nakamura,Pookie,10 mai 1995 ou 10 mai 1995Bamako,,,Malienne,


In [101]:
dfs_chart[2020]["Fevrier"]['Naissance'].apply(lambda row : test(row))

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Name: Naissance, Length: 100, dtype: object

In [45]:
months = {
    "janvier": "01",
    "février": "02", 
    "mars": "03", 
    "avril": "04", 
    "mai": "05", 
    "juin": "06", 
    "juillet": "07", 
    "août": "08", 
    "septembre": "09", 
    "octobre": "10", 
    "novembre": "11", 
    "décembre": "12",
    "1er": "1"}

In [8]:
#chercher mois et faire traitement
def insertNaissance2(row):
    texte = re.sub("[\(\[].*?[\)\]]", "", row['Naissance'])
    texte = texte.split()
    #print(texte)
    res = "0000-01-01"
    if len(texte)>2 and texte[0].isnumeric() and texte[2].isnumeric():
        res = str(texte[2])+"-"+str(months[texte[1]])+"-"+str(texte[0])
    return res

In [81]:
def pays(row):
    if row["Pays"] != " ":
        return row["Pays"]
    elif row["Nationalité"] != " ":
        return row["Nationalité"]
    elif row["Origine"] != " ":
        return row["Origine"]
    elif row["Pays d'origine"] != " ":
        return row["Pays d'origine"]
    else:
        texte = re.sub("[\(\[].*?[\)\]]", "", row['Naissance'])
        texte = texte.split()
        if len(texte)>3:
            return texte[-1]
        else:
            return ""

## Recherche Naissance

In [63]:
from datetime import datetime
def insertNaissance(row):
    match = re.match('([1-2][0-9]{3})', row)
    print(match)
    if match is not None:
        match = re.sub('ou.*', "", match.group())
        match = " ".join(months.get(ele, ele) for ele in match.split())
        match = match.replace(" ", "/" )
        match = datetime.strptime(match, '%y')
 
    return match

In [133]:

for years in dfs_chart:
     for table in dfs_chart[years].values():
          to_insert = table.drop(['Rank', 'Music'], axis=1)
          to_insert['Naissance_to_insert'] = to_insert.apply(lambda row : insertNaissance(row), axis=1)
          #to_insert['Pays'] = to_insert['Pays'].astype("string")
          #to_insert['Pays_to_insert'] = to_insert.apply(lambda row : pays(row), axis=1)
          #print(to_insert['Pays_to_insert'].unique())

## Insert artist et music

In [45]:
artist = pd.read_csv("../etapes/4 - cleanning wikipedia results/nationalité_pays/artist.csv", encoding="utf-8-sig")

In [52]:
cur = conn.cursor()
cur.execute("truncate music cascade;")
cur.close()

In [60]:
def insertNaissance(row):
    if not pd.isna(row) and re.search(r"([1-2][0-9]{3})",row) is not None:
        return  re.search(r"([1-2][0-9]{3})",row).group()
    else:
        return row

In [61]:
cur = conn.cursor()
for ind in artist.index:
    print(artist["Naissance"][ind])
    command = (""" INSERT INTO artist(nameArtist, birth, country) VALUES ( %s,%s,%s) """)
    cur.execute(command, (artist["Artist"][ind], insertNaissance(artist["Naissance"][ind]), artist["Nationalité"][ind]))
cur.close()

nan
nan
2 mai 1982 (40 ans)le plessis-bouchard, val-d'oise,  france
26 février 1971 (51 ans)six-fours-les-plages (france)
24 mai 1980 (42 ans)nogent-sur-marne, (val-de-marne),  france
nan
28 septembre 1987 (35 ans)houston, texas,  états-unis
26 juin 1972 (50 ans)sherbrooke, québec (canada)
19 septembre 1977 (45 ans)suresnes,  france
nan
nan
nan
nan
20 juin 1981 (41 ans)bastia (haute-corse)
30 octobre 1982 (40 ans)melun (france)
28 septembre 1982 (40 ans)saint-renan, finistère, france
4 septembre 1981 (41 ans)houston, texas (états-unis)
11 mai 1983 (39 ans)panama
6 juillet 1975 (47 ans)[1]queens, new york, états-unis
25 juillet 1980 (42 ans)nicosie, (chypre)
6 juin 1978 (44 ans)mantes-la-jolie, yvelines, france
10 février 1981 (41 ans)bathurst
9 janvier 1970 (52 ans)etterbeek, belgique
nan
22 octobre 1968 (54 ans)kingston,  jamaïque
nan
9 janvier 1973 (49 ans)[2],[3]kingston, jamaïque
2 décembre 1981 (41 ans)mccomb, mississippi,  états-unis
19 août 1982 (40 ans)melun (seine-et-marne)
19

In [69]:
cur = conn.cursor()
cur.execute("ROLLBACK")
cur.close()

In [51]:
cur = conn.cursor()
cur.execute("""SELECT * FROM artist""")
la = cur.fetchall()
for row in la:
    print(row)
cur.close()

(1, 1, 'sexy pour moi', None, 1920, 2)


In [192]:
cursor = conn.cursor()
cursor.execute(""" SELECT table_name FROM information_schema.tables WHERE table_schema='public' """)
for table in cursor.fetchall():
    print(table)

('artist',)
('music',)
('lyrics',)


In [62]:
music = pd.read_csv("../etapes/2 - cleanning chart/charts.csv", encoding="utf-8-sig")

In [70]:
cur = conn.cursor()
for ind in music.index:
    print(music["Music"][ind])
    command = (""" INSERT INTO music (title, releaseDate, month, ranking, id_artist) VALUES
    ( %s, %s, %s, %s,     (SELECT id_artist from artist WHERE nameArtist=%s)) """)
    cur.execute(command, (music["Music"][ind], int(music["Annee"][ind]), music["Mois"][ind], int(music["Rank"][ind]), music["Artist"][ind]))
cur.close()

sexy pour moi
l'orange - wot
week end
on n'oublie jamais rien, on vit avec
si demain... (turn around)
la bamba
in the shadows
so yesterday
reviens
c'est trop
love's divine
j'ai des choses a te dire
maria magdalena
ronde de nuit
sans (re)peres
illusion
ça va dech
be faithful
mon etoile
summer jam
ma rivale
toi, tu
hey oh
si j'avais su t'aimer
suivre une etoile
right on time
saturday night's alright for fight
baby boy
papito ven a mí
p.i.m.p.
incassable
je veux vivre
tant que c'est toi
bambina
going under
get my party on
numb
like glue
me against the music
number one (si tu veux)
where is the love?
dj
encore
vivre ça
on my way
we are the champions
si tu savais
we will rock you (remix)
j'attendrai
guilty
zinedine
être une femme (femme des années 80)
prendre racine
another one bites the dust / we will rock you
(dimmi) cosa ne sai
je n'ai jamais pleure
thoia thoing
never leave you (uh oooh, uh oooh)
slow
à contre-courant
la tactique aeo
can't hold us down
dancin'
ocean
rien ne me changera
o

## Genius

In [ ]:
pip install lyricsgenius --user

In [71]:
from lyricsgenius import Genius

genius = Genius("ETnXOiXHGWwb_KrkK7S44oezIt4HjJrar5oZ7qz_eNo3CkQ27br4oBWChpoKF0su")

In [75]:
music = pd.read_csv("../etapes/2 - cleanning chart/charts.csv", encoding="utf-8-sig")

In [76]:
music

,Mois,Annee,Rank,Artist,Music
0,Janvier,2004,1,tragedie,sexy pour moi
1,Janvier,2004,2,star academy,l'orange - wot
2,Janvier,2004,3,lorie pester,week end
3,Janvier,2004,4,hélène segara,"on n'oublie jamais rien, on vit avec"
4,Janvier,2004,5,kareen antonn,si demain... (turn around)
...,...,...,...,...,...
22660,Novembre,2022,96,timal,filtré
22661,Novembre,2022,97,tiakola,mode av
22662,Novembre,2022,98,orelsan,jour meilleur
22663,Novembre,2022,99,imagine dragons,bones


In [74]:
genius.search_song(title="something human", artist="Muse", song_id=None, get_full_info=False).lyrics

Searching for "something human" by Muse...
Done.


"TranslationsEspañolSomething Human Lyrics[Verse 1]\nMy circuits have blown\nI know it's self-imposed\nAnd all I have shared, and all I have loved\nIs all I'll ever own\nBut something has changed\nI feel so alive\nMy life just blew up, I'd give it all up\nI'll depressurize\n\n[Chorus]\nTen thousand miles left on the road\nFive-hundred hours 'til I am home\nI need something human, human\nHuman, human\n[Verse 2]\nLet's face all our fears\nCome out of the shade\nLet's burn all the money, absolve all the lies\nAnd wake up unscathed\nThe big picture's gone\nReplaced with visions of you\nNow life can begin, I've cleansed all my sins\nI'm about to break through\n\n[Chorus]\nFive thousand miles left on the road\nTwo-hundred hours 'til I am home\nI need something human, human\nHuman, human\n\n[Bridge]\nAnd I need the touch\nAnd something human, human\n\n[Chorus]\nLess than a mile left on the road\nI will be crawling through your door\nI need something human, human\nHuman, human\nYou might also 

In [80]:
data = []
for ind in music.index:
    lyric = genius.search_song(title=music["Music"][ind], artist=music["Artist"][ind], song_id=None, get_full_info=False)
    if lyric != None:
        data.append([music["Music"][ind], music["Artist"][ind], lyric])

Searching for "sexy pour moi" by tragedie...
Done.
Searching for "l'orange - wot" by star academy...
No results found for: 'l'orange - wot star academy'
Searching for "week end" by lorie pester...
Done.
Searching for "on n'oublie jamais rien, on vit avec" by hélène segara...
Done.
Searching for "si demain... (turn around)" by kareen antonn...
Done.
Searching for "la bamba" by star academy...
Done.
Searching for "in the shadows" by the rasmus...
Done.
Searching for "so yesterday" by hilary duff...
Done.
Searching for "reviens" by garou...
Done.
Searching for "c'est trop" by singuila...
Done.
Searching for "love's divine" by seal...
Done.
Searching for "j'ai des choses a te dire" by pearl...
Done.
Searching for "maria magdalena" by jessica marquez...
Done.
Searching for "ronde de nuit" by gomez...
Done.
Searching for "sans (re)peres" by sniper...
Done.
Searching for "illusion" by benassi bros...
Done.
Searching for "ça va dech" by caméra café...
Done.
Searching for "be faithful" by fat m

Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)

In [ ]:
lyrics = pd.DataFrame(data, columns=["Mois", "Annee", "Rank", "Artist", "Music"])

In [ ]:
lyrics.to_csv("../lyrics.csv", index=False, encoding="utf-8-sig")